In [ ]:
#Load libraries:
from data_statistics import *
%matplotlib inline

In [ ]:
all_data=False

In [ ]:
#Collect all data?
all_data=True

#Set up test time interval:
time_interval='4w' #2w

#Set up starting point, by default if will start from current time
starting_point=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#starting_point="2019-01-24 14:00:00"  # to set upl alternative starting point

title_tail=""
query_tail=""

if not all_data:
    ##tail to all the titles
    title_tail=" over the last "+time_interval+ " back  from "+ starting_point
    ##tail for all the influxdb queries
    query_tail=" AND time >= '"+starting_point+"'-"+time_interval

In [ ]:
if not all_data:#Set up test time interval:
    print("Time interval: ", time_interval)
    print("Starting point:",starting_point )
else:
    print("Selecting all data from database")

In [ ]:
#Set up influxdb connection:
client, client_df = connect_to_influxdb()

In [ ]:
min_sample_size=100
print("Minimum sample size:",min_sample_size)

## Download speed

In [ ]:
query_download = "SELECT * FROM SPEEDTEST_IPERF_DOWNLOAD WHERE PROVIDER!='iperf' AND DOWNLOAD>0"+ query_tail+";"
download_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_download,table_name='SPEEDTEST_IPERF_DOWNLOAD')

How many datapoints do we have?

In [ ]:
result=pd.DataFrame(download_df.groupby('SK_PI').size())
result.columns=['size']

In [ ]:
simple_bar_plot(xvalues=result.index.astype(int),
                yvalues=result['size'],
                name="downpload datapoints",
                title="Number of data points per device"+title_tail,
                ytitle="Number of datapoints")

Sort out only results that have more than min number of samples:

In [ ]:
tobe_excluded=result[result['size']<=min_sample_size].index
download_df=download_df[~download_df['SK_PI'].isin(tobe_excluded)]

In [ ]:
device_numbers_d=download_df['SK_PI'].unique()
device_numbers_d=list(map(int, device_numbers_d))
device_numbers_d= sorted(device_numbers_d)
print("Devices that are going to be statistically tested:",device_numbers_d)

### 1.Test if data is normally distributed:

#### Visual tests - Distribution plots

In [ ]:
t='Distribution plots for download speed per device'+title_tail
dist_subplots(dataframe=download_df,plot_value='DOWNLOAD',device_numbers=device_numbers_d, title=t)

#### Visual tests - QQ plots
Another popular plot for checking the distribution of a data sample is the quantile-quantile plot, Q-Q plot, or QQ plot for short.

This plot generates its own sample of the idealized distribution that we are comparing with, in this case the Gaussian distribution. The idealized samples are divided into groups (e.g. 5), called quantiles. Each data point in the sample is paired with a similar member from the idealized distribution at the same cumulative distribution.

The resulting points are plotted as a scatter plot with the idealized value on the x-axis and the data sample on the y-axis.

A perfect match for the distribution will be if  dots are as close as possible to the line.

In [ ]:
num_cols=4
num_rows=int(len(device_numbers_d)/num_cols)+1
fig = plt.figure(figsize = (15, num_rows*3), facecolor = "white")
i=1
for device in device_numbers_d:
    subset=download_df[download_df["SK_PI"]==device]
    ax = fig.add_subplot(num_rows, num_cols, i)
    p = qq_plot(subset['DOWNLOAD'], ax,device_number=device)
    i=i+1

fig.tight_layout()

#### Statistical tests

We are going to perform 3 statistical tests to check if data is normally distributed:
Shapiro-Wilk, Kolmogorov-Smirnov and D'Agostino's K-squared:
  
**H0** -  null hypothesis  - sample was drawn from a Gaussian(normal) distribution  
**Ha** - altenative hypothesis - sample was not drawn from a Gaussian(normal) distribution
we are going 95% confidence interval.
   
**Evaluation**:    
    p-value <= alpha: reject H0, not normal.  
    p-value > alpha: fail to reject H0, normal.

In [ ]:
alpha = 0.05

matrix_sw = []
for device in device_numbers_d:
    subset=download_df[download_df["SK_PI"]==device]
    shapiro_results =scipy.stats.shapiro(subset['DOWNLOAD'])
    ks_results = scipy.stats.kstest(subset['DOWNLOAD'], cdf='norm',args=(subset['DOWNLOAD'].mean(), subset['DOWNLOAD'].std()))
    dagostino_results = scipy.stats.normaltest(subset['DOWNLOAD'])
    matrix_sw.append(
    [device, len(subset['DOWNLOAD']) - 1, shapiro_results[0], shapiro_results[1], ks_results[0], ks_results[1],
    dagostino_results[0], dagostino_results[1]])

df = pd.DataFrame(matrix_sw)

df.columns=['Device number', 'Degrees of freedom', 'Shapiro-Wilk Test Statistic', 'Shapiro-Wilk p-value',
     'Kolmogorov-Smirnov test Statistic','Kolmogorov-Smirnov p-value',
     "D'Agostino's K-squared test Statistic","D'Agostino's K-squared p-value" ]

df=df.reset_index().set_index("Device number")
df.drop(['index'], axis=1, inplace=True)
df["Shapiro-Wilk"]='n'
df.loc[(df["Shapiro-Wilk p-value"]>alpha),"Shapiro-Wilk"]='y'
df["Kolmogorov-Smirnov"]='n'
df.loc[(df["Kolmogorov-Smirnov p-value"]>alpha),"Kolmogorov-Smirnov"]='y'
df["D'Agostino's K-squared"]='n'
df.loc[(df["D'Agostino's K-squared p-value"]>alpha),"D'Agostino's K-squared"]='y'

result=df[["Shapiro-Wilk","Kolmogorov-Smirnov","D'Agostino's K-squared"]]
result_table = ff.create_table(result, index=True)
iplot(result_table)

Comparing with visualizations, Kolmogorov-Smirinov test looks right.

In [ ]:
normally_distributed_d=list(result[result["Kolmogorov-Smirnov"]=='y'].index)
not_normally_distributed_d=list(result[result["Kolmogorov-Smirnov"]=='n'].index)
print("Normally distributed devices according to the Kolmogorov-Smirnov test: ",normally_distributed_d)

### 2.Test if mean of the population of normally distributed data statistically different from the threshold of 50Mbps:
A t-test is commonly used to determine whether the mean of a population significantly differs from a specific value (called the hypothesized mean) or from the mean of another population.

For normally distributed devices  - how far are they from the treshold of 50Mbps?   
**H0** - null hypothesis - mean is = 50 (less than 50?)  
**Ha** - altenative hypothesis -mean is greater than 50  

Confidence interval 95%

**1-tailed 1-sample t-test**:  
p/2 <= alpha: reject H0    
t-statistics> 0 and p/2 > alpha: fail to reject H0, mean is greater than 50  

In [ ]:
true_mean = 50
alpha = 0.05
list_t = []
for device in normally_distributed_d:
#for device in device_numbers_d:
    subset=download_df[download_df["SK_PI"]==device]
    onesample_results = scipy.stats.ttest_1samp(subset["DOWNLOAD"], true_mean)
    list_t.append(
    [device, round(subset["DOWNLOAD"].mean(),2), round(subset["DOWNLOAD"].std(),2), round(onesample_results[0],2), onesample_results[1]/2])

df1 = pd.DataFrame(list_t)

df1.columns=['Device number',"Mean","Standart deviation","t-statistics","p-value/2"]
df1=df1.reset_index().set_index("Device number")
df1 = df1.drop('index', 1)
df1["Statistically > 50Mbps"]='n'
df1.loc[(df1["t-statistics"]>0) & (df1["p-value/2"]< alpha),"Statistically > 50Mbps"]='y'

result_table = ff.create_table(df1, index=True)
iplot(result_table)
#for device in normally_distributed_d:
#    subset=download_df[download_df["SK_PI"]==device]
#    onesample_results = scipy.stats.ttest_1samp(subset["DOWNLOAD"], true_mu)
#   print("Device number: ", device)
#    print("Mean: ",subset["DOWNLOAD"].mean())
#    print("Standart deviation: ",subset["DOWNLOAD"].std())
#    print(" ",onesample_results)
#    if (onesample_results[0] > 0) & (onesample_results[1]/2 < alpha):
#        print("Reject the null hypothesis, mean is greater than {}".format(true_mu))
#    else:
#        print("Accept the null hypothesis")
#    print("")

In [ ]:
t="Download speed by device "+title_tail
#simple_boxplot(dataframe=download_df,plot_value='DOWNLOAD',sort_value='SK_PI',
simple_boxplot(dataframe=download_df[download_df["SK_PI"].isin(normally_distributed_d)],plot_value='DOWNLOAD',sort_value='SK_PI',
               title=t, 
               ytitle="Download speed (Mbps)",
               xtitle="Device number", downloadline=True, jitter=True)

### 3.Test if data that is not normally distributed  statistically different from the threshold of 50Mbps


In [ ]:
print("Devices with data not normally distributed: ",not_normally_distributed_d)

If the download speed distribution is not normal, the mean has an unknown
distribution and strictly speaking the t-test is inapplicable. However according to
the central limit theorem, as the sample size increases, the distribution of the
mean tends to be normal. Therefore if the sample size is big enough, the t-test
and confidence interval are valid even if the download speed is not from a normal
distribution.

We will create 500 samples with replacement with 45 values each and calculate mean for every sample:
(https://drive.google.com/file/d/1SuOqkR38tB5f5-iUINXKgQvh871f-jjC/view) SamKnows recommends using 1000 samples (sample size 45)

In [ ]:
sample_size_r=45
num_samples_r=500
matrix_r=[]
for device in not_normally_distributed_d:
    list_r=[]
    subset=download_df[download_df["SK_PI"]==device]
    for i in range(num_samples_r):
        sample = resample(subset["DOWNLOAD"], replace=True, n_samples=sample_size_r, random_state=i)
        list_r.append(sample.mean())
    matrix_r.append([device]+list_r)
    
df_r = pd.DataFrame(matrix_r)
df_r=df_r.transpose()
df_r.columns = df_r.iloc[0]
df_r=df_r.reindex(df_r.index.drop(0))
df_r.head()

#### Visual check for resampled means: QQplots

In [ ]:
num_cols=4
num_rows=int(len(not_normally_distributed_d)/num_cols)+1
fig = plt.figure(figsize = (15, num_rows*3), facecolor = "white")
i=1
for device in not_normally_distributed_d:
    subset=df_r[device]
    ax = fig.add_subplot(num_rows, num_cols, i)
    p = qq_plot(subset, ax,device_number=device)
    i=i+1

fig.tight_layout()

#### Statistical checks for resampled means

In [ ]:
alpha = 0.05

matrix_sw = []
for device in not_normally_distributed_d:
    subset=df_r[device]
    shapiro_results =scipy.stats.shapiro(subset)
    ks_results = scipy.stats.kstest(subset, cdf='norm',args=(subset.mean(), subset.std()))
    dagostino_results = scipy.stats.normaltest(subset)
    matrix_sw.append(
    [device, len(subset) - 1, shapiro_results[0], shapiro_results[1], ks_results[0], ks_results[1],
     dagostino_results[0], dagostino_results[1]])

df = pd.DataFrame(matrix_sw)

df.columns=['Device number', 'Degrees of freedom', 'Shapiro-Wilk Test Statistic', 'Shapiro-Wilk p-value',
     'Kolmogorov-Smirnov test Statistic','Kolmogorov-Smirnov p-value',
     "D'Agostino's K-squared test Statistic","D'Agostino's K-squared p-value" ]

df=df.reset_index().set_index("Device number")
df.drop(['index'], axis=1, inplace=True)
df["Shapiro-Wilk"]='n'
df.loc[(df["Shapiro-Wilk p-value"]>alpha),"Shapiro-Wilk"]='y'
df["Kolmogorov-Smirnov"]='n'
df.loc[(df["Kolmogorov-Smirnov p-value"]>alpha),"Kolmogorov-Smirnov"]='y'
df["D'Agostino's K-squared"]='n'
df.loc[(df["D'Agostino's K-squared p-value"]>alpha),"D'Agostino's K-squared"]='y'

result=df[["Shapiro-Wilk","Kolmogorov-Smirnov","D'Agostino's K-squared"]]
result_table = ff.create_table(result, index=True)
iplot(result_table)

Almost all of the resampled data loooks normaly distribured according to the Kolmogorov-Smirnov test (except for device #2)

In [ ]:
normally_distributed_d1=list(result[result["Kolmogorov-Smirnov"]=='y'].index)
print("Normally distributed devices according to the Kolmogorov-Smirnov test: ",normally_distributed_d1)

### t-tests for resampled means

In [ ]:
true_mean = 50
alpha = 0.05
list_t = []
for device in normally_distributed_d1:
    subset=df_r[device]
    onesample_results = scipy.stats.ttest_1samp(subset, true_mean)
    list_t.append(
    [device, round(subset.mean(),2), round(subset.std(),2), round(onesample_results[0],2), onesample_results[1]/2])

df2 = pd.DataFrame(list_t)

df2.columns=['Device number',"Mean","Standart deviation","t-statistics","p-value/2"]
df2=df2.reset_index().set_index("Device number")
df2 = df2.drop('index', 1)
df2["Statistically > 50Mbps"]='n'
df2.loc[(df2["t-statistics"]>0) & (df2["p-value/2"]< alpha),"Statistically > 50Mbps"]='y'

result_table = ff.create_table(df2, index=True)
iplot(result_table)

In [ ]:
t="Download speed by device "+title_tail
#simple_boxplot(dataframe=download_df,plot_value='DOWNLOAD',sort_value='SK_PI',
simple_boxplot(dataframe=download_df[download_df["SK_PI"].isin(not_normally_distributed_d)],plot_value='DOWNLOAD',sort_value='SK_PI',
               title=t, 
               ytitle="Download speed (Mbps)",
               xtitle="Device number", downloadline=True, jitter=True)

### Combining the results

In [ ]:
df_result=df2.append(df1)
df_result.sort_index(inplace=True)
#df_result

In [ ]:
download_df["above50"]=0
download_df.loc[(download_df["DOWNLOAD"]>50),"above50"]=1

In [ ]:
summary_download=download_df.groupby("SK_PI")['above50'].count().reset_index()
summary_download.rename(columns={'above50':'download_count'}, inplace=True)

mean_download=round(download_df.groupby("SK_PI")['DOWNLOAD'].mean(),2).reset_index()
mean_download.rename(columns={'DOWNLOAD':'download_mean'}, inplace=True)

std_download=round(download_df.groupby("SK_PI")['DOWNLOAD'].std(),2).reset_index()
std_download.rename(columns={'DOWNLOAD':'download_std'}, inplace=True)

subset_above50=download_df[download_df['above50']==1]
summary_above50=subset_above50.groupby("SK_PI")['above50'].count().reset_index()
summary_above50.rename(columns={'above50':'download_above50'}, inplace=True)

summary_download=pd.merge(summary_download, summary_above50,  how='outer', left_on=['SK_PI'], right_on = ['SK_PI'])
summary_download=pd.merge(summary_download, mean_download,  how='outer', left_on=['SK_PI'], right_on = ['SK_PI'])
summary_download=pd.merge(summary_download, std_download,  how='outer', left_on=['SK_PI'], right_on = ['SK_PI'])

summary_download.fillna(0, inplace=True)
summary_download['Download speeds more than 50  Mbps  (%)']=round(summary_download['download_above50']/summary_download['download_count']*100)
summary_download=summary_download.sort_values(by=['SK_PI'], ascending=[True])
summary_download["Device number"]=summary_download["SK_PI"]
summary_download=summary_download.reset_index().set_index("Device number")
summary_download.drop(['index','SK_PI'], axis=1, inplace=True)
#summary_download[["download_mean","download_std","Download speeds less than 50  Mbps  (%)"]]

In [ ]:
summary_download=summary_download.join(df_result,how="outer")
summary_download[["download_mean","download_std","Download speeds more than 50  Mbps  (%)","Statistically > 50Mbps"]]

In [ ]:
t="Download speed by device "+title_tail
simple_boxplot(dataframe=download_df,plot_value='DOWNLOAD',sort_value='SK_PI',
               title=t, 
               ytitle="Download speed (Mbps)",
               xtitle="Device number", downloadline=True, jitter=True)

Its questionable for device 14 - 57%of datapoints is above 50Mbs, mean of the sample is 92Mbps and statistical tests show that mean of the population is more than 50Mbs as well.

## Ping latency(speedtest)
We are checking data that is coming from speedtest  - so it has the same number of datapoints as download speed. We will exclude devices with number of datapoints less than minimum sample size.

In [ ]:
query_ping = "SELECT * FROM SPEEDTEST_IPERF_PING WHERE PROVIDER!='iperf' AND PING>0"+ query_tail+";"
ping_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_ping,table_name='SPEEDTEST_IPERF_PING')

In [ ]:
ping_df = ping_df[ping_df.PING != 1800000.000]

In [ ]:
result=pd.DataFrame(ping_df.groupby('SK_PI').size())
result.columns=['size']

In [ ]:
#simple_bar_plot(xvalues=result.index.astype(int),
#                yvalues=result['size'],
#                name="ping datapoints",
#                title="Number of data points per device"+title_tail,
#                ytitle="Number of datapoints")

In [ ]:
tobe_excluded=result[result['size']<=min_sample_size].index
ping_df=ping_df[~ping_df['SK_PI'].isin(tobe_excluded)]
device_numbers_p=ping_df['SK_PI'].unique()
device_numbers_p=list(map(int, device_numbers_p))
device_numbers_p= sorted(device_numbers_p)
print("Devices that are going to be statistically tested:",device_numbers_p)

### 1.Test if data is normally distributed:

In [ ]:
t='Distribution plots for ping latency per device'+title_tail
dist_subplots(dataframe=ping_df,plot_value='PING',device_numbers=device_numbers_p, title=t)

In [ ]:
num_cols=4
num_rows=int(len(device_numbers_p)/num_cols)+1
fig = plt.figure(figsize = (15, num_rows*3), facecolor = "white")
i=1
for device in device_numbers_p:
    subset=ping_df[ping_df["SK_PI"]==device]
    ax = fig.add_subplot(num_rows, num_cols, i)
    p = qq_plot(subset['PING'], ax,device_number=device)
    i=i+1

fig.tight_layout()

In [ ]:
matrix_sw = []

for device in device_numbers_p:
    subset=ping_df[ping_df["SK_PI"]==device]
    shapiro_results =scipy.stats.shapiro(subset['PING'])
    ks_results = scipy.stats.kstest(subset['PING'], cdf='norm',args=(subset['PING'].mean(),subset['PING'].std()))
    dagostino_results = scipy.stats.normaltest(subset['PING'])

    matrix_sw.append(
    [device, len(subset['PING']) - 1, shapiro_results[0], shapiro_results[1], ks_results[0], ks_results[1],
     dagostino_results[0], dagostino_results[1]])

df = pd.DataFrame(matrix_sw)

df.columns=['Device number', 'Degrees of freedom', 'Shapiro-Wilk Test Statistic', 'Shapiro-Wilk p-value',
     'Kolmogorov-Smirnov test Statistic','Kolmogorov-Smirnov p-value',
     "D'Agostino's K-squared test Statistic","D'Agostino's K-squared p-value" ]    
    
df=df.reset_index().set_index("Device number")
#df.drop(['index'], axis=1, inplace=True)
df["Shapiro-Wilk"]='n'
df.loc[(df["Shapiro-Wilk p-value"]>alpha),"Shapiro-Wilk"]='y'
df["Kolmogorov-Smirnov"]='n'
df.loc[(df["Kolmogorov-Smirnov p-value"]>alpha),"Kolmogorov-Smirnov"]='y'
df["D'Agostino's K-squared"]='n'
df.loc[(df["D'Agostino's K-squared p-value"]>alpha),"D'Agostino's K-squared"]='y'

result=df[["Shapiro-Wilk","Kolmogorov-Smirnov","D'Agostino's K-squared"]]
result_table = ff.create_table(result, index=True)
iplot(result_table)

In [ ]:
normally_distributed_p=list(result[result["Kolmogorov-Smirnov"]=='y'].index)
not_normally_distributed_p=list(result[result["Kolmogorov-Smirnov"]=='n'].index)
print("Normally distributed devices according to the Kolmogorov-Smirnov test: ",normally_distributed_p)

### 2.Test if mean of normally distributed data statistically different from the mean of data coming  from control device #3:

For normally distributed devices  - how far means are from the control dataset? 

**H0** - null hypothesis - means are equal   
**Ha** - altenative hypothesis - means are not equal  
confidence interval -95%

**2-tailed 2-sample t-test**:  
p <= alpha: reject H0(means are equal)  
p > alpha: fail to reject H0, means are not equal  

In [ ]:
control_subset=ping_df[ping_df["SK_PI"]==3]
alpha = 0.05
list_t = []
for device in normally_distributed_p:
    subset=ping_df[ping_df["SK_PI"]==device]
    onesample_results = scipy.stats.ttest_ind(control_subset["PING"],subset["PING"], equal_var=False)
    list_t.append(
    [device, round(subset["PING"].mean(),2), round(subset["PING"].std(),2), round(onesample_results[0],2), onesample_results[1]])

df1 = pd.DataFrame(list_t)

df1.columns=['Device number',"Mean","Standart deviation","t-statistics","p-value"]
df1=df1.reset_index().set_index("Device number")
df1 = df1.drop('index', 1)
df1["Statistically different to CD"]='n'
df1.loc[(df1["p-value"]< alpha),"Statistically different to CD"]='y'

result_table = ff.create_table(df1, index=True)
#df1
iplot(result_table)
#alpha = 0.05
#control_subset=ping_df[ping_df["SK_PI"]==3]
#print("Control subset device number: ", device)
#print("Control subset mean: ",control_subset["PING"].mean())
#print("Control subset standart deviation: ",control_subset["PING"].std())
#print("")

#for device in normally_distributed_p:
#    subset=ping_df[ping_df["SK_PI"]==device]
#    results = scipy.stats.ttest_ind(control_subset["PING"],subset["PING"], equal_var=False)
#    print("Device number: ", device)
#    print("Mean: ",subset["PING"].mean())
#    print("Standart deviation: ",subset["PING"].std())
#    print(" ",results)
#    if (results[1] < alpha):
#        print("Reject the null hypothesis, means are not equal")
#    else:
#        print("Accept the null hypothesis,means are equal")
 #   print("")

# pvalue <<< 0.05 reject null hypothesis, weight mean between male and female are not equal

In [ ]:
normally_distributed_p.append(3)
t="Ping latency  by device "+title_tail
simple_boxplot(dataframe=ping_df[ping_df["SK_PI"].isin(normally_distributed_p)],plot_value='PING',sort_value='SK_PI',
               title=t, 
               ytitle="Ping latency (Miliseconds)",
               xtitle="Device number")

### 3.Test if mean of not normally distributed data statistically different from the mean of data coming  from control device #3:

In [ ]:
print("Devices with data not normally distributed: ",not_normally_distributed_p)

In [ ]:
sample_size_r=45
num_samples_r=500
matrix_r=[]
for device in not_normally_distributed_p:
    list_r=[]
    subset=ping_df[ping_df["SK_PI"]==device]
    for i in range(num_samples_r):
        sample = resample(subset["PING"], replace=True, n_samples=sample_size_r, random_state=i)
        list_r.append(sample.mean())
    matrix_r.append([device]+list_r)
    
df_r = pd.DataFrame(matrix_r)
df_r=df_r.transpose()
df_r.columns = df_r.iloc[0]
df_r=df_r.reindex(df_r.index.drop(0))
df_r.head()

In [ ]:
alpha = 0.05

matrix_sw = []
for device in not_normally_distributed_p:
    subset=df_r[device]
    shapiro_results =scipy.stats.shapiro(subset)
    ks_results = scipy.stats.kstest(subset, cdf='norm',args=(subset.mean(), subset.std()))
    dagostino_results = scipy.stats.normaltest(subset)
    matrix_sw.append(
    [device, len(subset) - 1, shapiro_results[0], shapiro_results[1], ks_results[0], ks_results[1],
     dagostino_results[0], dagostino_results[1]])

df = pd.DataFrame(matrix_sw)

df.columns=['Device number', 'Degrees of freedom', 'Shapiro-Wilk Test Statistic', 'Shapiro-Wilk p-value',
     'Kolmogorov-Smirnov test Statistic','Kolmogorov-Smirnov p-value',
     "D'Agostino's K-squared test Statistic","D'Agostino's K-squared p-value" ]

df=df.reset_index().set_index("Device number")
df.drop(['index'], axis=1, inplace=True)
df["Shapiro-Wilk"]='n'
df.loc[(df["Shapiro-Wilk p-value"]>alpha),"Shapiro-Wilk"]='y'
df["Kolmogorov-Smirnov"]='n'
df.loc[(df["Kolmogorov-Smirnov p-value"]>alpha),"Kolmogorov-Smirnov"]='y'
df["D'Agostino's K-squared"]='n'
df.loc[(df["D'Agostino's K-squared p-value"]>alpha),"D'Agostino's K-squared"]='y'

result=df[["Shapiro-Wilk","Kolmogorov-Smirnov","D'Agostino's K-squared"]]
result_table = ff.create_table(result, index=True)
iplot(result_table)

In [ ]:
num_cols=4
num_rows=int(len(not_normally_distributed_d)/num_cols)+1
fig = plt.figure(figsize = (15, num_rows*3), facecolor = "white")
i=1
for device in not_normally_distributed_d:
    subset=df_r[device]
    ax = fig.add_subplot(num_rows, num_cols, i)
    p = qq_plot(subset, ax,device_number=device)
    i=i+1

fig.tight_layout()

In [ ]:
normally_distributed_p1=list(result[result["Kolmogorov-Smirnov"]=='y'].index)
print("Normally distributed devices(resampled means) according to the Kolmogorov-Smirnov test: ",normally_distributed_p1)

In [ ]:
control_subset=df_r[3]
alpha = 0.05
list_t = []
for device in normally_distributed_p1:
    subset=df_r[device]
    onesample_results = scipy.stats.ttest_ind(control_subset,subset, equal_var=False)
    list_t.append(
    [device, round(subset.mean(),2), round(subset.std(),2), round(onesample_results[0],2), onesample_results[1]])

df2 = pd.DataFrame(list_t)

df2.columns=['Device number',"Mean","Standart deviation","t-statistics","p-value"]
df2=df2.reset_index().set_index("Device number")
df2 = df2.drop('index', 1)
df2["Statistically different to CD"]='n'
df2.loc[(df2["p-value"]< alpha),"Statistically different to CD"]='y'

result_table = ff.create_table(df2, index=True)
iplot(result_table)

Device 3 only statistically equal to itself:

In [ ]:
normally_distributed_p.append(3)
t="Ping latency  by device "+title_tail
simple_boxplot(dataframe=ping_df[ping_df["SK_PI"].isin(not_normally_distributed_p)],plot_value='PING',sort_value='SK_PI',
               title=t, 
               ytitle="Ping latency (Miliseconds)",
               xtitle="Device number")